# Task 2


Importing all the libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from string import punctuation
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec, KeyedVectors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, BertModel
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Loading the training dataset

In [ ]:
train_data = pd.read_csv('/content/Task-2-test-dataset3.csv')

Prerocessing for Naive Bayes Approach

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
processed_df = train_data.copy()

def preprocess_text_bayes(un_processed_data):
    tokens = word_tokenize(un_processed_data)
    tokenized_words = []

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    for token in tokens:
        # Checking if the word is alphabetic
        if token.isalpha() and token.lower() not in stop_words and token.lower() not in string.punctuation:
            # Converting the word to lowercase
            tokenized_words.append(lemmatizer.lemmatize(token.lower()))

    preprocessed_data = ' '.join(tokenized_words)
    return preprocessed_data

processed_df['tokenized_synopsis_bayes'] = processed_df['plot_synopsis'].apply(preprocess_text_bayes)

Prerocessing for LSTM Approach

In [ ]:
def preprocess_text_lstm(un_processed_data):
    tokens = word_tokenize(un_processed_data)
    tokenized_words = []

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    for token in tokens:
        # Checking if the word is alphabetic
        if token.isalpha() and token.lower() not in stop_words and token.lower() not in string.punctuation:
            # Converting the word to lowercase
            tokenized_words.append(lemmatizer.lemmatize(token.lower()))

    preprocessed_data = ' '.join(tokenized_words)
    return preprocessed_data

processed_df['tokenized_synopsis_lstm'] = processed_df['plot_synopsis'].apply(preprocess_text_lstm)

Arranging the training datset

In [ ]:
# Separating the text and labels
train_text = processed_df["tokenized_synopsis_bayes"]
train_labels = train_data[["comedy", "cult", "flashback", "historical", "murder", "revenge", "romantic", "scifi", "violence"]]

KeyError: "None of [Index(['comedy', 'cult', 'flashback', 'historical', 'murder', 'revenge',\n       'romantic', 'scifi', 'violence'],\n      dtype='object')] are in the [columns]"

Create a TfidfVectorizer to convert the text into numerical features

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
train_features = tfidf_vectorizer.fit_transform(train_text)
# print(train_features.shape)

Training the Multinomial Naive Bayes classifier

In [ ]:
multinomial_nb = MultinomialNB(alpha=0.01)
one_vs_rest_classifier = OneVsRestClassifier(multinomial_nb)
nb_classifier = OneVsRestClassifier(MultinomialNB)
# Creating the model
nb_classifier_ =  make_pipeline(tfidf_vectorizer, one_vs_rest_classifier)
nb_classifier_.fit(train_text, train_labels)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('onevsrestclassifier',
                 OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01)))])

loading the processing the Validation Data

In [ ]:
processed_df_validation = pd.read_csv('./data/Task-2-validation-dataset.csv')
processed_df_validation['tokenized_bayes'] = processed_df_validation['plot_synopsis'].apply(preprocess_text_bayes)

In [ ]:
import pandas as pd
from google.colab import files

def apply_threshold(probabilities, threshold=0.65):
    return (probabilities >= threshold).astype(int)

prediction_train_probability_validation = nb_classifier_.predict_proba(processed_df_validation['tokenized_bayes'])
prediction_with_custom_threshold = apply_threshold(prediction_train_probability_validation, threshold=0.23)


document_ids = processed_df_validation["ID"].tolist()
res = []
for idx, label_probs in enumerate(prediction_with_custom_threshold):
    document_id = document_ids[idx]
    res.append((document_id, *label_probs))

res_df = pd.DataFrame(res)


# Save the DataFrame to a CSV file without including the index and column names
res_df.to_csv('10949863-Task2-method-a-validation.csv', index=False, header=False)

# Download the file to your local machine
files.download('10949863-Task2-method-a-validation.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

loading the processing the Test Data

In [ ]:
processed_df_test = pd.read_csv('./data/Task-2-test-dataset1.csv')
processed_df_test['tokenized_bayes'] = processed_df_test['plot_synopsis'].apply(preprocess_text_bayes)

In [ ]:
import pandas as pd
from google.colab import files

def apply_threshold(probabilities, threshold=0.65):
    return (probabilities >= threshold).astype(int)

prediction_train_probability_test = nb_classifier_.predict_proba(processed_df_test['tokenized_bayes'])
prediction_with_custom_threshold = apply_threshold(prediction_train_probability_test, threshold=0.23)

document_ids = processed_df_test["ID"].tolist()
res = []
for idx, label_probs in enumerate(prediction_with_custom_threshold):
    document_id = document_ids[idx]
    res.append((document_id, *label_probs))

res_df = pd.DataFrame(res)


# Save the DataFrame to a CSV file without including the index and column names
res_df.to_csv('10949863-Task2-method-a-test.csv', index=False, header=False)

# Download the file to your local machine
files.download('10949863-Task2-method-a-test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LSTM Approach

In [ ]:
# Separate the text and labels
train_text = train_data["plot_synopsis"]
train_labels = train_data[["comedy", "cult", "flashback", "historical", "murder", "revenge", "romantic", "scifi", "violence"]]

Tokenizing and padding the sequences

In [ ]:
max_words = 3000  # 4000 len(vocabulary)
max_len = 200
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_text)
sequences = tokenizer.texts_to_sequences(train_text)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

Building the LSTM model

In [ ]:
model_lstm_ = Sequential()
model_lstm_.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_lstm_.add(LSTM(100))
model_lstm_.add(Dense(9, activation='sigmoid'))
model_lstm_.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm_.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          384000    
                                                                 
 lstm (LSTM)                 (None, 100)               91600     
                                                                 
 dense (Dense)               (None, 9)                 909       
                                                                 
Total params: 476509 (1.82 MB)
Trainable params: 476509 (1.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Training the model_lstm_

In [ ]:
model_lstm_.fit(padded_sequences, train_labels, epochs= 10, batch_size = 42) # bs = 42 epoch = 10 best

Epoch 1/10
197/197 [==============================] - 70s 336ms/step - loss: 0.4761 - accuracy: 0.2446
Epoch 2/10
197/197 [==============================] - 68s 344ms/step - loss: 0.4564 - accuracy: 0.2598
Epoch 3/10
197/197 [==============================] - 66s 337ms/step - loss: 0.4395 - accuracy: 0.2715
Epoch 4/10
197/197 [==============================] - 67s 340ms/step - loss: 0.4174 - accuracy: 0.2978
Epoch 5/10
197/197 [==============================] - 68s 344ms/step - loss: 0.3931 - accuracy: 0.3280
Epoch 6/10
197/197 [==============================] - 66s 335ms/step - loss: 0.3705 - accuracy: 0.3613
Epoch 7/10
197/197 [==============================] - 67s 341ms/step - loss: 0.3522 - accuracy: 0.3844
Epoch 8/10
197/197 [==============================] - 68s 344ms/step - loss: 0.3352 - accuracy: 0.4104
Epoch 9/10
197/197 [==============================] - 66s 333ms/step - loss: 0.3192 - accuracy: 0.4233
Epoch 10/10
197/197 [==============================] - 67s 342ms/step - l

Loading and processing the Validation Data

In [ ]:
processed_df_validation = pd.read_csv('./data/Task-2-validation-dataset.csv')
processed_df_validation['tokenized_lstm'] = processed_df_validation['plot_synopsis'].apply(preprocess_text_lstm)

Tokenizing and padding the sequences for the validation data

In [ ]:
# Tokenizing and padding the sequences for the validation data
validation_text = processed_df_validation['tokenized_lstm']

validation_sequences = tokenizer.texts_to_sequences(validation_text)
padded_validation_sequences = pad_sequences(validation_sequences, maxlen=max_len, padding='post')

val_predictions = model_lstm_.predict(padded_validation_sequences)
val_predictions_binary = (val_predictions > 0.32).astype(int)


document_ids = processed_df_validation["ID"].tolist()
res = []
for idx, label_probs in enumerate(val_predictions_binary):
    document_id = document_ids[idx]
    res.append((document_id, *label_probs))

res_df = pd.DataFrame(res)
res_df.to_csv('10949863-Task2-method-b-validation.csv', index=False, header=False)
files.download('10949863-Task2-method-b-validation.csv')

38/38 [==============================] - 3s 71ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading and processing the Test Data

In [ ]:
processed_df_test = pd.read_csv('./data/Task-2-test-dataset1.csv')
processed_df_test['tokenized_lstm'] = processed_df_test['plot_synopsis'].apply(preprocess_text_lstm)

Tokenizing and padding the sequences for the validation data

In [ ]:
test_text = processed_df_test['tokenized_lstm']

test_sequences = tokenizer.texts_to_sequences(test_text)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding='post')

test_predictions = model_lstm_.predict(padded_test_sequences)
test_predictions_binary = (test_predictions > 0.32).astype(int)


document_ids = processed_df_test["ID"].tolist()
res = []
for idx, label_probs in enumerate(test_predictions_binary):
    document_id = document_ids[idx]
    res.append((document_id, *label_probs))

res_df = pd.DataFrame(res)
res_df.to_csv('10949863-Task2-method-b-test.csv', index=False, header=False)
files.download('10949863-Task2-method-b-test.csv')

38/38 [==============================] - 5s 135ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>